In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add the repo root (two levels up from this notebook) to sys.path
sys.path.insert(0, os.path.abspath("../../"))

In [23]:
import courselib.utils.loaders as loaders

df = loaders.load_music_30_sec()
#df = loaders.load_music_3_sec()

Loading from `features_30_sec.csv`...


In [20]:
from courselib.utils.splits import train_test_split

# Extract only MFCC columns from df
mfcc_columns = [col for col in df.columns if col.startswith('mfcc')]
features = mfcc_columns + ['label']

# Do train test split
X, Y, X_train, Y_train, X_test, Y_test = train_test_split(
    df[features],
    training_data_fraction=0.8,
    class_column_name='label',
    shuffle=True,
    return_numpy=True
)

print('Training data split as follows:')
print(f'  Training data samples: {len(X_train)}')
print(f'      Test data samples: {len(X_test)}')

Training data split as follows:
  Training data samples: 7992
      Test data samples: 1998


In [19]:
from courselib.models.multiclass_svm import KernelMulticlassOvR

svmOvR = KernelMulticlassOvR(kernel='rbf', sigma=1.0)
svmOvR.fit(X_train, Y_train)

KeyboardInterrupt: 

In [14]:
svmOvR.evaluate_models(X_test, Y_test)
svmOvR.evaluate_accuracy(X_test, Y_test)

📊 Accuracy of each binary model (One-vs-Rest):
  - Class 'blues': 89.5000 (Support vectors: 800)
  - Class 'classical': 89.0000 (Support vectors: 800)
  - Class 'country': 91.5000 (Support vectors: 800)
  - Class 'disco': 89.5000 (Support vectors: 800)
  - Class 'hiphop': 88.0000 (Support vectors: 800)
  - Class 'jazz': 91.5000 (Support vectors: 800)
  - Class 'metal': 91.5000 (Support vectors: 800)
  - Class 'pop': 90.0000 (Support vectors: 800)
  - Class 'reggae': 91.0000 (Support vectors: 800)
  - Class 'rock': 90.0000 (Support vectors: 800)
🎯 Overall accuracy (OvR): 0.1050


In [21]:
from courselib.models.multiclass_svm import KernelMulticlassOvO

svmOvO = KernelMulticlassOvO(kernel='rbf', sigma=1.0)
svmOvO.fit(X_train, Y_train)

In [22]:
svmOvO.evaluate_models(X_test, Y_test)
svmOvO.evaluate_accuracy(X_test, Y_test)

📊 Accuracy of each OvO binary classifier:
  - Classifier 'blues' vs 'classical': 51.1166 (Support vectors: 1595)
  - Classifier 'blues' vs 'country': 49.4975 (Support vectors: 1599)
  - Classifier 'blues' vs 'disco': 49.8741 (Support vectors: 1602)
  - Classifier 'blues' vs 'hiphop': 51.2626 (Support vectors: 1602)
  - Classifier 'blues' vs 'jazz': 51.1688 (Support vectors: 1615)
  - Classifier 'blues' vs 'metal': 54.3860 (Support vectors: 1601)
  - Classifier 'blues' vs 'pop': 51.7857 (Support vectors: 1608)
  - Classifier 'blues' vs 'reggae': 51.4139 (Support vectors: 1611)
  - Classifier 'blues' vs 'rock': 47.4699 (Support vectors: 1583)
  - Classifier 'classical' vs 'country': 49.3857 (Support vectors: 1588)
  - Classifier 'classical' vs 'disco': 50.9852 (Support vectors: 1591)
  - Classifier 'classical' vs 'hiphop': 52.3457 (Support vectors: 1591)
  - Classifier 'classical' vs 'jazz': 47.7157 (Support vectors: 1604)
  - Classifier 'classical' vs 'metal': 55.3922 (Support vectors: 